## 1. PromptTemplate

### 1. PromptTemplate

使用str.format()补全template

In [4]:
from langchain.prompts import PromptTemplate

promt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
promt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

### 2. ChatPromptTemplate

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

message = chat_template.format_messages(name="Bob",
                                        user_input="what is your name?")
message

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='what is your name?')]

In [7]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


### 3. PromptTemplate&LCEL

In [8]:
prompt_val = promt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

In [9]:
prompt_val.to_string()

'Tell me a funny joke about chickens.'

In [10]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funny joke about chickens.')]

### 4. ChatPromptTemplate&LCEL

In [11]:
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])

In [13]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."

In [12]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

## 2. Prompt Composition

### 1. String prompt composition

In [15]:
from langchain.prompts import PromptTemplate

prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [18]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

#### 2. Chat Prompt composition

In [24]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

prompt = SystemMessage(content="You are a nice pirate")

new_prompt = (
    prompt + HumanMessage(content="hi")
    + AIMessage(content="what?")
    + "{input}"
)
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

## 3. Example Selecttor 
**TODO：Example Selector Types**

#### 1. Select by length

In [29]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_langth=25,
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [30]:
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


## 4. Partial prompt templates

#### 1. Partial with strings

In [32]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))

foobaz


In [33]:
# 初始化template时就定义要部分填充的变量
prompt = PromptTemplate(
    template="{foo}{bar}",
    input_variables=["bar"],
    partial_variables={"foo": "foo"}
)

print(prompt.format(bar="baz"))

foobaz


#### 2. Partial with functions

In [37]:
from datetime import datetime

def _get_datetime():
    now = datetime.now()
    return now.strftime("%m-%d-%Y, %H:%M:%S")

prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day: {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day: 02-17-2024, 17:48:53


In [38]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 02-17-2024, 17:49:09


## 6. Pipeline Prompt
+ Final prompt: 最后返回的prompt
+ Pipeline prompts: 传入一个”步骤“列表，这个元组包含两部分：一个字符串名（string name）和一个提示模板（prompt template）, string name 是一个简短的标识符

In [62]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [63]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [64]:
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

In [65]:
pipeline_prompt.input_variables

['example_a', 'person', 'input', 'example_q']

In [66]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:
